# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Import required packages

In [1]:
## LIST OF ALL IMPORTS

import csv
import math
import random
import time
import os.path as path
from datetime import datetime

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cv2

import tensorflow as tf
# from sklearn.decomposition import PCA
from sklearn.utils import shuffle

from tensorflow.contrib.layers import flatten

# Visualizations will be shown in the notebook.
%matplotlib inline 

repeat=0

---
## Step 1: Load Datasets

In [2]:
## LOAD PICKLED DATASET & SPLIT DATA

t0=time.clock() # Obtain run-times.
print("Obtaining datasets.")
# Training, Validation, and Testing data.
training_file='traffic-signs-data/train.p' 
validation_file='traffic-signs-data/valid.p'
testing_file='traffic-signs-data/test.p'
augment_file='traffic-signs-data/augmented_train.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']


print("Ensuring equal lengths for features and labels.")
assert(len(X_train)==len(y_train))
assert(len(X_valid)==len(y_valid))
assert(len(X_test)==len(y_test))

# Dataset labels in the German traffic sign dataset.
label_legend='signnames.csv'
arr_classes=pd.read_csv(label_legend,index_col=None).values
arr_classes=arr_classes[:,1]
print("All datasets loaded.")

Obtaining datasets.
Ensuring equal lengths for features and labels.
All datasets loaded.


---

## Step 2: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [3]:
## BASIC UNDERSTANDING OF DATASET

# Number of training examples.
n_train = X_train.shape[0]

# Number of validation examples.
n_validation = X_valid.shape[0]

# Number of testing examples.
n_test = X_test.shape[0]

# Shape of an traffic sign image.
image_shape = X_train[1].shape

# Unique classes/labels in the dataset.
n_classes = len(np.unique(y_train))

# Further manipulation.
total_sets=n_train+n_validation+n_test
frac_train=n_train/total_sets
frac_valid=n_validation/total_sets
frac_test=n_test/total_sets

class_list,class_indices,class_counts=np.unique(y_train, return_index=True, return_counts=True)

print("Class indices", class_indices)
print("Class counts", class_counts)


print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Number of testing examples =", n_test)
print("There are",total_sets, "datasets, split",round(frac_train,2),"-",\
      round(frac_valid,2),"-",round(frac_test,2),"training, validation, and testing respectively.")
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Class indices [ 9960  2220 31439  5370  6810 12360 21450 23730 15870 11040 17130  8580
 27329 21810 29219 29909  5010 30449 20370  6630 25950 25680  4500  1770
 10800 33449  1230 10350 26849 10560 25020   210 10140 26250 20010 18930
   900  4830 14010 25410  4200     0  9750]
Class counts [ 180 1980 2010 1260 1770 1650  360 1290 1260 1320 1800 1170 1890 1920  690
  540  360  990 1080  180  300  270  330  450  240 1350  540  210  480  240
  390  690  210  599  360 1080  330  180 1860  270  300  210  210]
Number of training examples = 34799
Number of validation examples = 4410
Number of testing examples = 12630
There are 51839 datasets, split 0.67 - 0.09 - 0.24 training, validation, and testing respectively.
Image data shape = (32, 32, 3)
Number of classes = 43


### Exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

#### Visualizing sample training set images

#### Training and Testing set representations

----

## Step 3: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Augment the Dataset

In [4]:
max_bincount=max(class_counts)
min_bincount=min(class_counts)
print("This step tries to smooth out possible wrong classifications and bias due to varying class lengths as the maximum class size is {} and the minimum is {}.".format(max_bincount,min_bincount))

This step tries to smooth out possible wrong classifications and bias due to varying class lengths as the maximum class size is 2010 and the minimum is 180.


### Preprocess the Dataset

The dataset is preprocessed by gray-scale conversion followed by normalizing to have zero mean and equal variance:'(pixel - 128)/ 128', then shuffling the datasets.


In [5]:
## DATA PRE-PROCESS FUNCTIONS
if (path.exists(augment_file) and repeat==0):
    pass
else:
    def grayscale(raw_image): # Convert to grayscale
        return cv2.cvtColor(raw_image,cv2.COLOR_BGR2GRAY)

    def normalize(raw_image): # Min-Max normalization
        min_pixel=np.min(raw_image)
        max_pixel=np.max(raw_image)
        return ((raw_image-min_pixel)/(max_pixel-min_pixel))
    
    def equalize(raw_image): # Adaptive Histogram equalization
        clahe=cv2.createCLAHE()
        return clahe.apply(raw_image)

    def randomize(dataset,labels):
        permutation=np.random.permutation(labels.shape[0])
        shuffled_dataset=dataset[permutation,:,:]
        shuffled_labels=labels[permutation,:,:]
        return (shuffled_dataset,shuffled_labels)    

In [6]:
if (path.exists(augment_file) and repeat==0):
    pass
else:
    print("Preprocessing started.")

    X_train_normalized=[]
    X_valid_grayscale=[]
    X_test_grayscale=[]
    for image in X_train:
        train_image_grayscale=equalize(grayscale(image))
        X_train_normalized.append(normalize(train_image_grayscale))
    for image in X_valid:
        valid_image_grayscale=equalize(grayscale(image))
        X_valid_grayscale.append(normalize(valid_image_grayscale))
    for image in X_test:
        test_image_grayscale=equalize(grayscale(image))
        X_test_grayscale.append(normalize(test_image_grayscale))
    print("Preprocessing: Grayscaling, Histogram Equalization, and Normalizing complete.")

Preprocessing started.
Preprocessing: Grayscaling, Histogram Equalization, and Normalizing complete.


### Augment the Dataset

Increasing class sizes and images for each class in the training set by various image manipulations. Also aims to obtain a balanced representation of classes.

In [7]:
## IMAGE MANIPULATION FUNCTIONS
if (path.exists(augment_file) and repeat==0):
    pass
else:
    def rotation(raw_image,angle_range):
        theta_rot=np.random.uniform(angle_range)-0.5*angle_range
        rows,cols=raw_image.shape
        rot_M=cv2.getRotationMatrix2D((cols/2,rows/2),theta_rot,1)
        return cv2.warpAffine(raw_image,rot_M,(cols,rows))

    def translation(raw_image,translation_range):
        rows,cols=raw_image.shape
        trans_x=translation_range*np.random.uniform()-0.5*translation_range
        trans_y=translation_range*np.random.uniform()-0.5*translation_range
        trans_M=np.float32([[1,0,trans_x],[0,1,trans_y]])
        return cv2.warpAffine(raw_image,trans_M,(cols,rows))

    def shear(raw_image):
        rows,cols=raw_image.shape
        x1=0.2*cols;y1=0.2*rows
        x2=0.8*cols;y2=0.8*rows
        mult_x=np.random.random(3)*cols
        mult_y=np.random.random(3)*rows

        points_1=np.float32([[y1,x1],
                             [y2,x1],
                            [y1,x2]])

        points_2=np.float32([[y1+mult_y[0],x1+mult_x[0]],
                             [y2+mult_y[1],x1+mult_x[1]],
                            [y1+mult_y[2],x2+mult_x[2]]])

        shear_M=cv2.getAffineTransform(points_1,points_2)
        return cv2.warpAffine(raw_image,shear_M,(cols,rows))

    def scale(raw_image):
        rows,cols=raw_image.shape
        scale_factor=np.random.uniform(0.5,5.0)
        return cv2.resize(raw_image,dsize=(32,32),fx=scale_factor,fy=scale_factor,interpolation=cv2.INTER_CUBIC)

    # def contrast(raw_image): # Removed because we are now working with a 1-channel image
    #     rows,cols=raw_image.shape
    #     hsv=cv2.cvtColor(raw_image,cv2.COLOR_BGR2HSV)
    #     h_channel,s_channel,v_channel=cv2.split(hsv)
    #     h_channel=np.add(h_channel,random.uniform(-100,100))
    #     v_channel=np.add(v_channel,random.uniform(-100,100))
    #     merged=np.uint8(np.dstack((h_channel,s_channel,v_channel)))
    #     return cv2.cvtColor(merged,cv2.COLOR_HSV2BGR)

    def flip_secondary(X,Y):
        class_list,class_indices,class_counts=np.unique(Y, return_index=True, return_counts=True)
        x_flip_out=np.array([0,32,32,3])
        y_flip_out=np.array([0])

        label_flip_vertical=np.array([9,10,11,12,13,15,17,18,21,22,23,25,26,\
                                      27,28,29,30,31,32,35,40,41,42])
        label_flip_horizontal=np.array([1,5,7,9,10,12,15,17,32,38,39,40,41,42])
        label_flip_classes=np.array([[19,20],
                                     [20,19],
                                     [33,34],
                                     [34,33],
                                     [36,37],
                                     [37,36],
                                     [38,39],
                                     [39,38]]) #Source label (original label), target/goal label

        for class_label in range(43):
            
            X_flip_out=np.append(X_flip_out,X[Y==class_label],axis=0)
            
            if class_label in label_flip_vertical:
                X_flip_out=np.append(X_flip_out,cv2.flip(X[y==class_label],1),axis=0)
            print("Images in vertically-invariant classes have been flipped.")
            
            if class_label in label_flip_horizontal:
                X_flip_out=np.append(X_flip_out,cv2.flip(X[y==class_label],0),axis=0)
            print("Images in horizontally-invariant classes have been flipped.")
            
            if class_label in label_flip_classes[:,0]:
                target_class=label_flip_classes[label_flip_classes[:,0]==class_label][0,1]
                X_flip_out=np.append(X_flip_out,cv2.flip(X[y==target_class],1),axis=0)
            
            print("Images in ambiguous classed have been flipped.")
            extended_length=X_flip_out[0]-y_flip_out[0]
            y_flip_out=np.append(y_flip_out,np.full((extended_length),class_label))
        
#         for index in label_flip_vertical: # Previous implementation that was streamlined
#             temp_imageindex=class_indices[index]
#             temp_classcount=class_counts[index]
#             temp_images=[]
#             v_flipped_images=[]
#             v_flipped_labels=[]
#             for i in range (temp_imageindex,temp_imageindex+temp_classcount):
#                 temp_images.append(X[i])
#             for image in temp_images:
#                 v_flipped_images.append(cv2.flip(image,1))
#                 v_flipped_labels.append(index)
#             x_flip_out=np.array(v_flipped_images)
#             y_flip_out=np.array(v_flipped_labels)

#         for index in label_flip_horizontal:
#             temp_imageindex=class_indices[index]
#             temp_classcount=class_counts[index]
#             temp_images=[]
#             h_flipped_images=[]
#             h_flipped_labels=[]
#             for i in range (temp_imageindex,temp_imageindex+temp_classcount):
#                 temp_images.append(X[i])
#             for image in temp_images:
#                 h_flipped_images.append(cv2.flip(image,0))
#                 h_flipped_labels.append(index)
#             x_flip_out=np.append(x_flip_out,np.array(h_flipped_images),axis=0)
#             y_flip_out=np.append(y_flip_out,np.array(h_flipped_labels),axis=0)

    #     for flip_map in label_flip_classes:
    #         actual=flip_map[0]
    #         desired=flip_map[1]
    #         temp_imageindex=class_indices[actual]
    #         temp_classcount=class_counts[actual]
    #         temp_images=[]
    #         both_images=[]
    #         both_labels=[]
    #         for i in range(temp_imageindex,temp_imageindex+temp_classcount):
    #             temp_images.append(X[i])
    #         for image in temp_images:
    #             both_images.append(image)
    #             both_labels.append(desired)

    #         x_flip_out=np.append(x_flip_out,np.array(both_images),axis=0)
    #         y_flip_out=np.append(y_flip_out,np.array(both_labels),axis=0)
        print("Augmenting images by flipping has been completed.")
        return (x_flip_out,y_flip_out)

In [ ]:
if (path.exists(augment_file) and repeat==0):
    pass
else:
    X_existing_flipped,y_existing_flipped=flip_secondary(X_train_normalized,y_train)
    print("Flip complete.")

In [ ]:
if (path.exists(augment_file) and repeat==0):
    pass
else:
    indices_augmented=[]
    X_train_augmented=np.copy(X_existing_flipped)
    y_train_augmented=np.copy(y_existing_flipped)

    balance_threshold=850
    for class_index in range(0,n_classes):
        print("Skipping images with label {}".format(class_index))
        image_index=np.where(y_train==class_index)
        class_size=(np.size(image_index))

        repeat=balance_threshold-class_size
        if class_size<=balance_threshold:
            for i in range(0,repeat):
                if (i%100==0):
                    print("Class Label ", class_index,"--> Class Image",i)
                indices_augmented.append(X_train_augmented.shape[0])
                augment_raw_copy=X_train_normalized[image_index[0][i%class_size]]
                a0=rotation(augment_raw_copy,40)
                a1=translation(augment_raw_copy,10)
                a2=shear(augment_raw_copy)
                a3=scale(augment_raw_copy)
                
                X_train_augmented=np.concatenate((X_train_augmented,[a0,a1,a2,a3]),axis=0)
                y_train_augmented=np.concatenate((y_train_augmented,[class_index,class_index,class_index,class_index]),axis=0)

    print("Complete augmenting has been done by linear transform (rotation, translation, shear, scale)")
    

0
Class Label  0 --> Class Image 0
Class Label  0 --> Class Image 100
Class Label  0 --> Class Image 200
Class Label  0 --> Class Image 300
Class Label  0 --> Class Image 400
Class Label  0 --> Class Image 500
Class Label  0 --> Class Image 600
1
2
3
4
5
6
Class Label  6 --> Class Image 0
Class Label  6 --> Class Image 100
Class Label  6 --> Class Image 200
Class Label  6 --> Class Image 300
Class Label  6 --> Class Image 400
7
8
9
10
11
12
13
14
Class Label  14 --> Class Image 0
Class Label  14 --> Class Image 100
15
Class Label  15 --> Class Image 0
Class Label  15 --> Class Image 100
Class Label  15 --> Class Image 200
Class Label  15 --> Class Image 300
16
Class Label  16 --> Class Image 0
Class Label  16 --> Class Image 100
Class Label  16 --> Class Image 200
Class Label  16 --> Class Image 300
Class Label  16 --> Class Image 400
17
18
19
Class Label  19 --> Class Image 0
Class Label  19 --> Class Image 100
Class Label  19 --> Class Image 200
Class Label  19 --> Class Image 300
Cl

In [ ]:
## PICKLING AUGMENTED DATASET
if (path.exists(augment_file) and repeat==0):
    pass
else:
    augmented_pickle='traffic-signs-data/augmented_train.p'

    try:
        print("Pickling dataset.")
        pickled=open(augmented_pickle,'wb')
        save = {
        'augmented_train_dataset': X_train_augmented,
        'augmented_train_labels': y_train_augmented,
        }
        pickle.dump(save,pickled)
        pickled.close()
    except:
        print("Error in creating a pickled dataset. Debug.")
    print("Dataset pickled.")
    del X_train_augmented
    del y_train_augmented

In [ ]:
with open(augment_file, mode='rb') as f:
    augment_train = pickle.load(f)

X_train_augmented, y_train_augmented = augment_train['augmented_train_dataset'], augment_train['augmented_train_labels']

# Plot traffic sign images.
print("AugmentX",X_train_augmented.shape)
print("AugmentY",y_train_augmented.shape)

# X_aug_shuffle,y_aug_shuffle=shuffle(X_train_augmented,y_train_augmented)

# aug_class_list,aug_class_indices,aug_class_counts=np.unique(y_aug_shuffle,return_index=True,return_counts=True)
# print("Class list", aug_class_list)
# print("Class Index", aug_class_indices)
# print("Class Counts", aug_class_counts)

# for CClass,IIndex,Counts in zip(augment_class_list,augment_class_indices,augment_class_counts):
#     print("Class {} {} : {} samples.".format(Class,arr_classes[Class],Counts))
#     Main=plt.figure(figsize=(10,5))
#     choice=random.sample(range(Index,Counts+Index),10)
#     for i in range (0,10):
#         row=Main.add_subplot(1,10,i+1,xticks=[],yticks=[])
#         row.imshow(X_flipped[choice[i]],cmap='gray')
#     plt.show()

# hist,bins=np.histogram(y_train_augmented,bins=n_classes)
# width=0.7*(bins[1]-bins[0])
# center=(bins[:-1]+bins[1:])/2
# plt.bar(center,hist,align='center',width=width)
# plt.show()
print("Done")

In [ ]:
## DATA EXPLORATION & VISUALIZATION

# Histogram plot to identify count of each class.
hist_ytrain_count=np.bincount(y_train)
hist_xtrain_count=len(hist_ytrain_count)

plt.figure(figsize=(12,5))
plt.hist(y_train_augmented, hist_xtrain_count, normed=False, facecolor='green',align='mid',rwidth=0.8,alpha=0.9,label='Augmented Training set')
plt.hist(y_train,hist_xtrain_count, normed=False, facecolor='black',align='mid',rwidth=0.8, alpha=0.75, label='Original Training set')
plt.legend(loc='upper right')
plt.xlabel('Class Label')
plt.ylabel('Count')
plt.title(r'Distribution of original training and augmented training dataset for each class')
plt.axis([0, 43,0,4000])
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
print("Shuffling and reshaping datasets")

X_train_shuffle,y_train_shuffle=shuffle(np.asarray(X_train_normalized).reshape(len(X_train_normalized),32,32,1),np.asarray(y_train).reshape(len(y_train),))
X_valid_shuffle,y_valid_shuffle=shuffle(np.asarray(X_valid_grayscale).reshape(len(X_valid_grayscale),32,32,1),np.asarray(y_valid).reshape(len(y_valid),))
X_test_shuffle,y_test_shuffle=shuffle(np.asarray(X_test_grayscale).reshape(len(X_test_grayscale),32,32,1),np.asarray(y_test).reshape(len(y_test),))

print("Shuffled training datasets shape: ",X_train_shuffle.shape,"& ",y_train_shuffle.shape)
print("Shuffled validation datasets shape: ",X_valid_shuffle.shape,"& ",y_valid_shuffle.shape)
print("Shuffled testing datasets shape: ",X_test_shuffle.shape,"& ",y_test_shuffle.shape)

print("Datasets ready for the neural network.")

### Model Architecture- Setups

In [ ]:
EPOCHS=50
BATCH_SIZE=100

# BASIC HYPERPARAMETERS
mu=0.0
sigma=0.1
learning_rate=0.001

In [ ]:
X=tf.placeholder(tf.float32,(None,32,32,1))
Y=tf.placeholder(tf.int32,(None))
keep_prob=tf.placeholder(tf.float32)
one_hot_y=tf.one_hot(Y,43)

### Architecture One: LeNet-5

In [ ]:
def LeNet_init(x):
    weights={
        'w_conv1': tf.Variable(tf.truncated_normal(shape=[4,4,1,6], mean=mu, stddev=sigma)),
        'w_conv2': tf.Variable(tf.truncated_normal(shape=[5,5,6,16], mean=mu, stddev=sigma)),
        'w_dense1': tf.Variable(tf.truncated_normal(shape=[400,120], mean=mu, stddev=sigma)),
        'w_dense2': tf.Variable(tf.truncated_normal(shape=[120,84], mean=mu, stddev=sigma)),
        'w_output': tf.Variable(tf.truncated_normal(shape=[84,43], mean=mu, stddev=sigma))
    }
    
    biases={
        'b_conv1': tf.Variable(tf.truncated_normal([6])),
        'b_conv2': tf.Variable(tf.truncated_normal([16])),
        'b_dense1': tf.Variable(tf.truncated_normal([120])),
        'b_dense2': tf.Variable(tf.truncated_normal([84])),
        'b_output': tf.Variable(tf.truncated_normal([43]))
    }

    mpool_filters={
        'filt_conv1': [1,2,2,1],
        'filt_conv2': [1,2,2,1]
    }

    Padding='VALID'
    Strides_conv=[1,1,1,1]
    Strides_pool=[1,2,2,1]
    
    # Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    conv1=tf.nn.conv2d(x,weights['w_conv1'],strides=Strides_conv,padding=Padding)
    conv1=tf.nn.bias_add(conv1,biases['b_conv1'])
    conv1=tf.nn.relu(conv1) # Activation.
    conv1=tf.nn.max_pool(conv1,ksize=mpool_filters['filt_conv1'],strides=Strides_pool,padding=Padding) # Pooling. 
    # Input = 28x28x6. Output = 14x14x6.
  
    # Layer 2: Convolutional. Output = 10x10x16.
    conv2=tf.nn.conv2d(conv1,weights['w_conv2'],strides=Strides_conv,padding=Padding)
    conv2=tf.nn.bias_add(conv2,biases['b_conv2'])
    conv2=tf.nn.relu(conv2) # Activation.
    conv2=tf.nn.max_pool(conv2,ksize=mpool_filters['filt_conv2'],strides=Strides_pool,padding=Padding) # Pooling.
    # Input = 10x10x16. Output = 5x5x16.
    
    # Flatten. Input = 5x5x16. Output = 400.
    lenet_flat=flatten(conv2)
    
    # Layer 3: Fully Connected. Input = 400. Output = 120.
    dense_1=tf.add(tf.matmul(lenet_flat,weights['w_dense1']),biases['b_dense1'])
    dense_1=tf.nn.relu(dense_1) # Activation.
#     full_one=tf.nn.dropout(full_one,keep_prob)
    
    # Layer 4: Fully Connected. Input = 120. Output = 84.
    dense_2=tf.add(tf.matmul(dense_1,weights['w_dense2']),biases['b_dense2'])
    dense_2=tf.nn.relu(dense_2) # Activation.
#     full_two=tf.nn.dropout(full_two,keep_prob)
    
    # Layer 5: Fully Connected. Input = 84. Output = 43.
    logits=tf.add(tf.matmul(dense_2,weights['w_output']),biases['b_output'])
 
    return logits    

#### Train, Validate and Test the Model: LeNet-5

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
logits_LeNet5=LeNet_init(X)
cross_entropy=tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y,logits=logits_LeNet5)
loss_operation=tf.reduce_mean(cross_entropy)
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
training_operation=optimizer.minimize(loss_operation)

In [ ]:
correct_prediction=tf.equal(tf.argmax(logits_LeNet5,1),tf.argmax(one_hot_y,1))
accuracy_operation=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
saver=tf.train.Saver()


def evaluate(X_dataset,Y_dataset):
    total_accuracy=0
    data_size=len(X_dataset)
    
    sess=tf.get_default_session()
    for offset in range(0,data_size,BATCH_SIZE):
        batch_x,batch_y=X_dataset[offset:offset+BATCH_SIZE],Y_dataset[offset:offset+BATCH_SIZE]
        local_accuracy=sess.run(accuracy_operation,feed_dict={X:batch_x, Y:batch_y, keep_prob:1.0})
        total_accuracy+=(local_accuracy*len(batch_x))
    return total_accuracy/data_size  

In [ ]:
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    data_size=len(X_train_shuffle)
    
    print("LeNet5 Training in progress...")
    print()
    
    for i in range(EPOCHS):
        X_train_shuffle,y_train_shuffle=shuffle(X_train_shuffle,y_train_shuffle)
        
        for offset in range(0,data_size,BATCH_SIZE):
            end=offset+BATCH_SIZE
            batch_x,batch_y=X_train_shuffle[offset:end],y_train_shuffle[offset:end]
            sess.run(training_operation, feed_dict={X:batch_x, Y:batch_y, keep_prob:0.50})
        
        validation_accuracy=evaluate(X_valid_shuffle,y_valid_shuffle)
        print("EPOCH {}...".format(i+1))
        print("Validation Accuracy of {:.5f}".format(validation_accuracy))
        print()
        
    saver.save(sess,'./lenet5_init')
    print("LeNet-5 model saved.")

### Architecture Two: VGG-16

In [ ]:
# Defining commonly used tensorflow functions

def convolution(layer,kernel,bias):
    # W- Weight [Filter height, Filter width, color_channels, k_output]
    temp=tf.nn.conv2d(layer,kernel,strides=[1,1,1,1],padding='SAME')
    return tf.nn.bias_add(temp,bias)

def full_connected(layer,weight,bias):
    temp=tf.matmul(layer,weight)
    return tf.nn.bias_add(temp,bias)
    
def maxpool(layer):
    return tf.nn.max_pool(layer,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

def maxpool_3x3(layer):
    return tf.nn.max_pool(layer,ksize=[1,3,3,1],strides=[1,1,1,1],padding='SAME')
    
def relu(layer):
    return tf.nn.relu(layer)

def dropout(layer,keep_prob):
    return tf.nn.dropout(layer,keep_prob)

In [ ]:
def VGG_16(x):
    weights={
        'w_conv1_1': tf.Variable(tf.truncated_normal(shape=[3,3,1,64], mean=mu, stddev=sigma)),
        'w_conv1_2': tf.Variable(tf.truncated_normal(shape=[3,3,64,64], mean=mu, stddev=sigma)),
        'w_conv2_1': tf.Variable(tf.truncated_normal(shape=[3,3,64,128], mean=mu, stddev=sigma)),
        'w_conv2_2': tf.Variable(tf.truncated_normal(shape=[3,3,128,128], mean=mu, stddev=sigma)),
        'w_conv3_1': tf.Variable(tf.truncated_normal(shape=[3,3,128,256], mean=mu, stddev=sigma)),
        'w_conv3_2': tf.Variable(tf.truncated_normal(shape=[3,3,256,256], mean=mu, stddev=sigma)),
        'w_conv3_3': tf.Variable(tf.truncated_normal(shape=[3,3,256,256], mean=mu, stddev=sigma)),
        'w_conv4_1': tf.Variable(tf.truncated_normal(shape=[3,3,256,512], mean=mu, stddev=sigma)),
        'w_conv4_2': tf.Variable(tf.truncated_normal(shape=[3,3,512,512], mean=mu, stddev=sigma)),
        'w_conv4_3': tf.Variable(tf.truncated_normal(shape=[3,3,512,512], mean=mu, stddev=sigma)),
        'w_conv5_1': tf.Variable(tf.truncated_normal(shape=[3,3,512,512], mean=mu, stddev=sigma)),
        'w_conv5_2': tf.Variable(tf.truncated_normal(shape=[3,3,512,512], mean=mu, stddev=sigma)),
        'w_conv5_3': tf.Variable(tf.truncated_normal(shape=[3,3,512,512], mean=mu, stddev=sigma)),
        'w_dense_1': tf.Variable(tf.truncated_normal(shape=[4608,4096], mean=mu, stddev=sigma)),
        'w_dense_2': tf.Variable(tf.truncated_normal(shape=[4096,4096], mean=mu, stddev=sigma)),
        'w_output': tf.Variable(tf.truncated_normal(shape=[4096,43], mean=mu, stddev=sigma))
        
    }
        
    biases={
        'b_conv1_1': tf.Variable(tf.truncated_normal(shape=[3,3,1,64], mean=mu, stddev=sigma)),
        'b_conv1_2': tf.Variable(tf.truncated_normal(shape=[3,3,64,64], mean=mu, stddev=sigma)),
        'b_conv2_1': tf.Variable(tf.truncated_normal(shape=[3,3,64,128], mean=mu, stddev=sigma)),
        'b_conv2_2': tf.Variable(tf.truncated_normal(shape=[3,3,128,128], mean=mu, stddev=sigma)),
        'b_conv3_1': tf.Variable(tf.truncated_normal(shape=[3,3,128,256], mean=mu, stddev=sigma)),
        'b_conv3_2': tf.Variable(tf.truncated_normal(shape=[3,3,256,256], mean=mu, stddev=sigma)),
        'b_conv3_3': tf.Variable(tf.truncated_normal(shape=[3,3,256,256], mean=mu, stddev=sigma)),
        'b_conv4_1': tf.Variable(tf.truncated_normal(shape=[3,3,256,512], mean=mu, stddev=sigma)),
        'b_conv4_2': tf.Variable(tf.truncated_normal(shape=[3,3,512,512], mean=mu, stddev=sigma)),
        'b_conv4_3': tf.Variable(tf.truncated_normal(shape=[3,3,512,512], mean=mu, stddev=sigma)),
        'b_conv5_1': tf.Variable(tf.truncated_normal(shape=[3,3,512,512], mean=mu, stddev=sigma)),
        'b_conv5_2': tf.Variable(tf.truncated_normal(shape=[3,3,512,512], mean=mu, stddev=sigma)),
        'b_conv5_3': tf.Variable(tf.truncated_normal(shape=[3,3,512,512], mean=mu, stddev=sigma)),
        'b_dense_1': tf.Variable(tf.truncated_normal(shape=[4608,4096], mean=mu, stddev=sigma)),
        'b_dense_2': tf.Variable(tf.truncated_normal(shape=[4096,4096], mean=mu, stddev=sigma)),
        'b_output': tf.Variable(tf.truncated_normal(shape=[4096,43], mean=mu, stddev=sigma))
        
    }

       
    # Block One: Two convolution layers 
    conv1_1=relu(convolution(layer,weights['w_conv1_1'],biases['b_conv1_1']))
    conv1_2=relu(convolution(conv1_1,weights['w_conv1_2'],biases['b_conv1_1']))
    
    # Block Two: One max pooling layer followed by Two convolution layers
    pool2=maxpool(conv1_2)
    conv2_1=relu(convolution(pool2,weights['w_conv2_1'],biases['b_conv2_1']))
    conv2_2=relu(convolution(conv2_1,weights['w_conv2_2'],biases['b_conv2_2']))
    
    # Block Three: One max pooling layer followed by Three convolution layers
    pool3=maxpool(conv2_2)
    conv3_1=relu(convolution(pool3,weights['w_conv3_1'],biases['b_conv3_1']))
    conv3_2=relu(convolution(conv3_1,weights['w_conv3_2'],biases['b_conv3_2']))
    conv3_3=relu(convolution(conv3_2,weights['w_conv3_3'],biases['b_conv3_3']))
    
    # Block Four: One max pooling layer followed by Three convolution layers
    pool4=maxpool(conv3_3)
    conv4_1=relu(convolution(pool4,weights['w_conv4_1'],biases['b_conv4_1']))
    conv4_2=relu(convolution(conv4_1,weights['w_conv4_2'],biases['b_conv4_2']))
    conv4_3=relu(convolution(conv4_2,weights['w_conv4_3'],biases['b_conv4_3']))
    
    # Block Five: One max pooling layer followed by Three convolution layers
    pool5=maxpool(conv4_3)
    conv5_1=relu(convolution(pool5,weights['w_conv5_1'],biases['b_conv5_1']))
    conv5_2=relu(convolution(conv5_1,weights['w_conv5_2'],biases['b_conv5_2']))
    conv5_3=relu(convolution(conv5_2,weights['w_conv5_3'],biases['b_conv5_3']))
    
    # Block Six: One max pooling layer followed by Three fully-connected layers
    pool6=maxpool(conv5_3)
    vgg_flat=flatten(pool6)
    dense6_1=relu(full_connected(vgg_flat,weights['w_dense_1'],biases['b_dense_1']))
    dense6_2=relu(full_connected(dense6_1,weights['w_dense_2'],biases['b_dense_2']))
    
    logits=full_connected(dense6_2,weights['w_output'],biases['b_output'])
    
    return logits    

#### Train, Validate and Test the Model: VGG-16

In [ ]:
logits_VGG16=VGG_16(X)
cross_entropy=tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y,logits=logits_VGG16)
loss_operation=tf.reduce_mean(cross_entropy)
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
training_operation=optimizer.minimize(loss_operation)

In [ ]:
correct_prediction=tf.equal(tf.argmax(logits_VGG16,1),tf.argmax(one_hot_y,1))
accuracy_operation=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
saver=tf.train.Saver()

In [ ]:
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    data_size=len(X_train_shuffle)
    
    print("VGG-16 Training in progress...")
    print()
    
    for i in range(EPOCHS):
        X_train_shuffle,y_train_shuffle=shuffle(X_train_shuffle,y_train_shuffle)
        
        for offset in range(0,data_size,BATCH_SIZE):
            end=offset+BATCH_SIZE
            batch_x,batch_y=X_train_shuffle[offset:end],y_train_shuffle[offset:end]
            sess.run(training_operation, feed_dict={X:batch_x, Y:batch_y, keep_prob:0.50})
        
        validation_accuracy=evaluate(X_valid_shuffle,y_valid_shuffle)
        print("EPOCH {}...".format(i+1))
        print("Validation Accuracy of {:.5f}".format(validation_accuracy))
        print()
        
    saver.save(sess,'./vgg16_init')
    print("VGG-16 model saved.")

### Architecture Three: Repurposed Barebones Model

In [ ]:
def inception_module_naive(layer,color_channels,kernel_output):
    mu=0
    sigma=0.1
    # 1*1 Convolution Layer
    filter_1_1=tf.Variable(tf.truncated_normal([1,1,color_channels,kernel_output], mu, sigma))
    block_conv1=tf.nn.conv2d(layer,filter_1_1,strides=[1,1,1,1], padding='SAME')
    
    # 3*3 Convolution Layer
    filter_3_1=tf.Variable(tf.truncated_normal([3,3,color_channels,kernel_output], mu, sigma))
    block_conv3=tf.nn.conv2d(layer,filter_3_1,strides=[1,1,1,1], padding='SAME')
    
    # 5*5 Convolution Layer
    filter_5_1=tf.Variable(tf.truncated_normal([5,5,color_channels,kernel_output], mu, sigma))
    block_conv5=tf.nn.conv2d(layer,filter_5_1,strides=[1,1,1,1], padding='SAME')
    
    # Average Pooling followed by 1*1 Convolution Layer
    average_pool=tf.nn.avg_pool(layer,ksize=[1,2,2,1],strides=[1,2,2,1], padding='SAME')
    
    filter_1_2=tf.Variable(tf.truncated_normal([1,1,color_channels,kernel_output],mu,sigma))
    block_avgpool_conv1=tf.nn.conv2d(average_pool,filter_1_2,strides=[1,1,1,1], padding='SAME')
    
    # Concatenating layers
    bias=tf.Variable(tf.truncated_normal([color_channels+3*kernel_output], mu, sigma))
    x=tf.concat([block_conv1,block_conv3,block_conv5,block_avgpool],axis=3)
    x=tf.nn.bias_add(x,bias)
    
    return tf.nn.relu(x)

In [ ]:
def inception_module_sequence1(layer,input_fmap,output_fmap):
    mu=0
    sigma=0.1
    
#     def convolution(layer,weight,bias):
        # W- Weight [Filter height, Filter width, color_channels, k_output]
#         temp=tf.nn.conv2d(layer,weight,strides=[1,1,1,1],padding='SAME')
#         return tf.nn.bias_add(temp,bias)
    
#     def maxpool_3x3(layer):
#         return tf.nn.max_pool(layer,ksize=[1,3,3,1],strides=[1,1,1,1],padding='SAME')
    
#     def relu(layer):
#         return tf.nn.relu(layer)
        
    # Weights,Biases
    # Three 1*1 Convolution Layers- Block One
    b1_conv_weight_1x1_1=tf.Variable(tf.truncated_normal([1,1,input_fmap,output_fmap],mu,sigma))
    b1_conv_bias_1x1_1=tf.Variable(tf.zeros([output_fmap]))
    
    b1_conv_weight_1x1_2=tf.Variable(tf.truncated_normal([1,1,input_fmap,0.5*output_fmap],mu,sigma))
    b1_conv_bias_1x1_2=tf.Variable(tf.zeros([0.5*output_fmap]))
    
    b1_conv_weight_1x1_3=tf.Variable(tf.truncated_normal([1,1,input_fmap,0.5*output_fmap],mu,sigma))
    b1_conv_bias_1x1_3=tf.Variable(tf.zeros([0.5*output_fmap]))
    
    # 3*3  Convolution Layer that follows b_1_conv_2- Block Two
    b2_conv_weight_3x3=tf.Variable(tf.truncated_normal([3,3,0.5*output_fmap,output_fmap],mu,sigma))
    c=tf.Variable(tf.zeros([output_fmap]))
    
    # 5*5 Convolution Layer that follows b1_conv_3- Block Two
    b2_conv_weight_5x5=tf.Variable(tf.truncated_normal([5,5,0.5*output_fmap,output_fmap],mu,sigma))
    b2_conv_bias_5x5=tf.Variable(tf.zeros([output_fmap]))
    
    # 1*1 Convolution Layer that follows a 3*3 maxpool layer- Block Two
    b2_conv_weight_1x1=tf.Variable(tf.truncated_normal([1,1,1,output_fmap],mu,sigma))
    b2_conv_bias_1x1=tf.Variable(tf.zeros([output_fmap]))
    
    
    # Fitting blocks
    # Parallel sub-modules in Block One
    b1_conv1_1x1=convolution(layer,b1_conv_weight_1x1_1,b1_conv_bias_1x1_1)
    b1_conv2_1x1=relu(convolution(layer,b1_conv_weight_1x1_2,b1_conv_bias_1x1_2))
    b1_conv3_1x1=relu(convolution(layer,b1_conv_weight_1x1_3,b1_conv_bias_1x1_2))
    b1_maxpool_3x3=maxpool_3x3(layer)
    
    # Parallel sub-modules in Block Two
    # 3*3 convolution connected to the second 1*1 convolution in Block One
    b2_conv1_3x3=convolution(b1_conv2_1x1,b2_conv_weight_3x3,b2_conv_weight_3x3)
    # 5*5 convolution connected to the third 1*1 convolution in Block One
    b2_conv1_5x5=convolution(b1_conv3_1x1,b2_conv_weight_5x5,b2_conv_bias_5x5)
    # 1*1 convolution connected to the max pool layer in Block One
    b2_conv1_1x1=convolution(b1_maxpool_3x3,b2_conv_weight_1x1,b2_conv_bias_1x1)
    
    # Concatenating modules b1_conv1_1x1 in Block One and all the modules in Block Two
    b3_concat=tf.concat([b1_conv1_1x1,b2_conv1_3x3,b2_conv1_3x3,b3_conv1_5x5,b2_conv1_1x1],3)
    
    inception=relu(b3_concat)
    
    return inception

In [ ]:
def Barebones_inception(x):
    weights={
        'w_conv_2x2_1': tf.Variable(tf.truncated_normal(shape=[2,2,1,6], mean=mu, stddev=sigma)),
        'w_conv_2x2_2': tf.Variable(tf.truncated_normal(shape=[5,5,6,16], mean=mu, stddev=sigma)),
        'w_conv_1x1_1': tf.Variable(tf.truncated_normal(shape=[1,1,16,32], mean=mu, stddev=sigma)),
        
        'w_dense1': tf.Variable(tf.truncated_normal(shape=[100352,120], mean=mu, stddev=sigma)),
        'w_dense2': tf.Variable(tf.truncated_normal(shape=[120,84], mean=mu, stddev=sigma)),
        'w_output': tf.Variable(tf.truncated_normal(shape=[84,43], mean=mu, stddev=sigma))
    }
    
    biases={
        'b_conv_2x2_1': tf.Variable(tf.truncated_normal([6])),
        'b_conv_2x2_2': tf.Variable(tf.truncated_normal([16])),
        'b_conv_1x1_1': tf.Variable(tf.truncated_normal([32])),
        
        'b_dense1': tf.Variable(tf.truncated_normal([120])),
        'b_dense2': tf.Variable(tf.truncated_normal([84])),
        'b_output': tf.Variable(tf.truncated_normal([43]))
    }
    
    keep_prob=tf.Variable(tf.float32)

    # Stem
    conv1_1=relu(convolution(layer,weights['w_conv_2x2_1'],biases['b_conv_2x2_1']))
    pool1_1=maxpool_3x3(conv1_1)
    conv1_2=relu(convolution(pool1_1,weights['w_conv_2x2_2'],biases['b_conv2_2x2_2']))
    conv1_3=relu(convolution(conv1_2,weights['w_conv_1x1_1'],biases['b_conv_1x1_1']))
    pool1_2=maxpool_3x3(conv1_3)
    
    # Body
    inception_s1=inception_module_sequence1(pool1_2,32,64)
    inception_s2=inception_module_sequence1(inception_s1,256,32)
    
    # Root
    barebones_flat=flatten(inception_s2)
    dense3_1=relu(full_connected(barebones_flat,weights['w_dense1'],biases['b_dense1']))
    dense3_1_dropout=dropout(dense3_1,keep_prob)
    dense3_2=relu(full_connected(dense3_1_dropout,weights['w_dense2'],biases['b_dense2']))
    dense3_2_dropout=dropout(dense3_2,keep_prob)
    
    logits=full_connected(dense3_2_dropout,weights['w_output'],biases['b_output'])
    
    return logits  

#### Train, Validate and Test the Model: Barebones inception architecture

In [ ]:
logits_bareinception=Barebones_inception(X)
cross_entropy=tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y,logits=logits_bareinception)
loss_operation=tf.reduce_mean(cross_entropy)

# global_variable=tf.Variable(0)
# learning_rate=
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
training_operation=optimizer.minimize(loss_operation)

In [ ]:
correct_prediction=tf.equal(tf.argmax(logits_VGG16,1),tf.argmax(one_hot_y,1))
accuracy_operation=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
saver=tf.train.Saver()

In [ ]:
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    data_size=len(X_train_shuffle)
    
    print("Simplified Inception model Training in progress...")
    print()
    
    for i in range(EPOCHS):
        X_train_shuffle,y_train_shuffle=shuffle(X_train_shuffle,y_train_shuffle)
        
        for offset in range(0,data_size,BATCH_SIZE):
            end=offset+BATCH_SIZE
            batch_x,batch_y=X_train_shuffle[offset:end],y_train_shuffle[offset:end]
            sess.run(training_operation, feed_dict={X:batch_x, Y:batch_y, keep_prob:0.50})
        
        validation_accuracy=evaluate(X_valid_shuffle,y_valid_shuffle)
        print("EPOCH {}...".format(i+1))
        print("Validation Accuracy of {:.5f}".format(validation_accuracy))
        print()
        
    saver.save(sess,'./bbinception_init')
    print("Barebones-Inception model saved.")

---

## Step 4: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.
print("Testing the CNN-model on new images obtained from the Belgian dataset.")
verification_folder='Belgian-traffic-signs-data/' 

folder_sample=random.sample(os.listdir(verification_folder),5)
images_path=[]
for folder in folder_sample:
    
    path=os.path.join(verification_folder,folder)
    items=os.listdir(path)
    valid_choice=[]
    image_array=[]
    
    for name in items:
        if name.endswith(".ppm"):
            valid_choice.append(name)
    image_sample=random.sample(valid_choice,3)
    
    for i in image_sample:
        image_path=os.path.join(path,i)
        images_path.append(image_path)
        
# Splitting functions for readability
images_list=np.zeros((15,32,32,3),dtype=np.uint8)
for i,image_index in enumerate(images_path):
    image=cv2.imread(image_index)
    images_list[i]=resize(image,(32,32))

# Plot, Subplots
plt.figure(figsize=(12,10))
for num_images in range(15):
    plt.subplot(5,3,num_images+1)
    plt.tight_layout()
    plt.imshow(images_list[num_images])
    plt.axis('off')
plt.show()      

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tk.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 5 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")

In [ ]:
sess.close()

In [ ]:
print("Script terminated at",str(datetime.now()))